<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Crawl_a_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index==0.10.57 llama-index-llms-openai openai==1.37.0 google-generativeai==0.5.4 newspaper3k==0.2.8 lxml_html_clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
USESCRAPER_API_KEY = "[USESCRAPER_API_KEY]"


There are two primary methods for extracting webpage content. The first method involves having a list of URLs; one can iterate through this list to retrieve the content of each page. The second method, web crawling, requires using a script or service to extract page URLs from a sitemap or manually following links on the page to access all the content. Initially, we will explore web scraping techniques before discussing how to use a service like usescraper.com to perform web crawling.


# 1. Scraping using `newspaper` Library


## Define URLs


In [3]:
urls = [
    "https://docs.llamaindex.ai/en/stable/understanding",
    "https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/",
    "https://docs.llamaindex.ai/en/stable/understanding/indexing/indexing/",
    "https://docs.llamaindex.ai/en/stable/understanding/querying/querying/",
]

## Get Page Contents


In [4]:
import newspaper

pages_content = []

# Retrieve the Content
for url in urls:
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append(
                {"url": url, "title": article.title, "text": article.text}
            )
    except:
        continue

In [5]:
pages_content[0]

{'url': 'https://docs.llamaindex.ai/en/stable/understanding',
 'title': 'Building an LLM Application',
 'text': "Building an LLM application#\n\nWelcome to the beginning of Understanding LlamaIndex. This is a series of short, bite-sized tutorials on every stage of building an LLM application to get you acquainted with how to use LlamaIndex before diving into more advanced and subtle strategies. If you're an experienced programmer new to LlamaIndex, this is the place to start.\n\nKey steps in building an LLM application#\n\nTip If you've already read our high-level concepts page you'll recognize several of these steps.\n\nThis tutorial has three main parts: Building a RAG pipeline, Building an agent, and Building Workflows, with some smaller sections before and after. Here's what to expect:\n\nUsing LLMs : hit the ground running by getting started working with LLMs. We'll show you how to use any of our dozens of supported LLMs, whether via remote API calls or running locally on your mac

In [6]:
len(pages_content)

4

## Convert to Document


In [7]:
from llama_index.core.schema import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(text=row["text"], metadata={"title": row["title"], "url": row["url"]})
    for row in pages_content
]

# 2. Submit the Crawler Job


In [8]:
import requests
import json

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/", # add your URLs here, e.g. "https://docs.llamaindex.ai/en/stable/understanding/"
]

payload = {
    "urls": urls_to_crawl,  # list of urls to crawl
    "output_format": "markdown",  # text, html, markdown
    "output_expiry": 604800,  # Automatically delete after X seconds
    "min_length": 50,  # Skip pages with less than X characters
    "page_limit": 3,  # Maximum number of pages to crawl
    "force_crawling_mode": "link",  # "link" follows links in the page reccursively, or "sitemap" to find pages from website's sitemap
    "block_resources": True,  # skip loading images, stylesheets, or scripts
    "include_linked_files": False,  # include files (PDF, text, ...) in output
}
headers = {
    "Authorization": "Bearer " + USESCRAPER_API_KEY,
    "Content-Type": "application/json",
}

response = requests.request(
    "POST", "https://api.usescraper.com/crawler/jobs", json=payload, headers=headers
)

response = json.loads(response.text)

print(response)

{'org': '2492', 'id': '7YDMC0KP6JPFYG0XH5SDADW0WV', 'urls': ['https://docs.llamaindex.ai/en/stable/understanding/'], 'exclude_globs': [], 'exclude_elements': 'nav, header, footer, script, style, noscript, svg, [role="alert"], [role="banner"], [role="dialog"], [role="alertdialog"], [role="region"][aria-label*="skip" i], [aria-modal="true"]', 'output_format': 'markdown', 'output_expiry': 604800, 'min_length': 50, 'page_limit': 10000, 'force_crawling_mode': 'link', 'block_resources': True, 'include_linked_files': False, 'createdAt': 1730468521773, 'status': 'starting', 'use_browser': True, 'sitemapPageCount': 0, 'notices': []}


## Get the Status


In [13]:
url = "https://api.usescraper.com/crawler/jobs/{}".format(response["id"])

status_res = requests.request("GET", url, headers=headers)

status_res = json.loads(status_res.text)

print(status_res["status"])
print(status_res["progress"])

running
{'scraped': 9, 'discarded': 0, 'failed': 1}


## Get the Data


In [14]:
url = "https://api.usescraper.com/crawler/jobs/{}/data".format(response["id"])

data_res = requests.request("GET", url, headers=headers)

data_res = json.loads(data_res.text)

print(data_res)

{'data': [{'meta': {'url': 'https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/', 'fetchedUrl': 'https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/', 'fetchedUrlStatusCode': 200, 'meta': {'title': 'Usage Pattern - LlamaIndex'}}, 'text': ' \n[ Skip to content ](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#usage-pattern)\n#Usage Pattern[#](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#usage-pattern)\n##Estimating LLM and Embedding Token Counts[#](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#estimating-llm-and-embedding-token-counts)\nIn order to measure LLM and Embedding token counts, you\'ll need to\n\n- Setup MockLLM and MockEmbedding objects\nfrom llama_index.core.llms import MockLLM\nfrom llama_index.core import MockEmbedding\n\nllm = MockLLM(max_tokens=256)\nembed_model 

In [15]:
print("URL:", data_res["data"][0]["meta"]["url"])
print("Title:", data_res["data"][0]["meta"]["meta"]["title"])
print("Content:", data_res["data"][0]["text"][0:500], "...")

URL: https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/
Title: Usage Pattern - LlamaIndex
Content:  
[ Skip to content ](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#usage-pattern)
#Usage Pattern[#](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#usage-pattern)
##Estimating LLM and Embedding Token Counts[#](https://docs.llamaindex.ai/en/stable/understanding/evaluating/cost_analysis/usage_pattern/#estimating-llm-and-embedding-token-counts)
In order to measure LLM and Embedding token counts, you'll need to
 ...


## Convert to Document


In [16]:
from llama_index.core.schema import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(
        text=row["text"],
        metadata={"title": row["meta"]["meta"]["title"], "url": row["meta"]["url"]},
    )
    for row in data_res["data"]
]

# Create RAG Pipeline


In [17]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0, model="gpt-4o-mini")

In [18]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [19]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [20]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [21]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [22]:
query_engine = index.as_query_engine()

In [23]:
res = query_engine.query("What is a query engine?")

In [24]:
res.response

'A query engine is a component that facilitates the process of querying by allowing users to send prompts to a language model (LLM) and receive responses. It operates by retrieving relevant documents from an index, processing those documents, and synthesizing a response based on the query and the retrieved data. The query engine can be customized to adjust retrieval strategies, post-processing steps, and response synthesis methods to enhance the querying experience.'

In [25]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("URL\t", src.metadata["url"])
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 2b0135eb-b0db-4597-b6f4-e99dec59eb20
Title	 Querying - LlamaIndex
URL	 https://docs.llamaindex.ai/en/stable/understanding/querying/querying/
Score	 0.5797795511788522
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 f5952992-33fe-4883-a264-996bb5062f61
Title	 Querying - LlamaIndex
URL	 https://docs.llamaindex.ai/en/stable/understanding/querying/querying/
Score	 0.503252077356493
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
